In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from pymystem3 import Mystem
nltk.download("stopwords")

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df = pd.read_excel("../case/02_Реальные_кейсы.xlsx")
df.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [23]:
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42, shuffle=True, stratify=df["Классификатор 1 уровня"].values)

In [20]:
russian_stopwords = stopwords.words("russian")
text_transformer = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=(1, 3), lowercase=True, max_features=150000)

In [24]:
%%time
X_train_text = text_transformer.fit_transform(train_df['Вопрос пользователя'])
X_test_text = text_transformer.transform(test_df['Вопрос пользователя'])

CPU times: user 14.6 ms, sys: 18 μs, total: 14.6 ms
Wall time: 14.3 ms


In [33]:
logit = LogisticRegression(C=5e1, solver='lbfgs', random_state=17, n_jobs=4)

In [34]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [41]:
%%time
cv_results = cross_val_score(logit, X_train_text, train_df['Классификатор 1 уровня'], cv=skf, scoring='accuracy')

/home/pc/Desktop/cp-russia-2024/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


CPU times: user 21.6 ms, sys: 2.02 ms, total: 23.7 ms
Wall time: 1.08 s


In [42]:
cv_results, cv_results.mean()

(array([0.59558824, 0.55882353, 0.61764706, 0.58823529, 0.52205882]),
 np.float64(0.5764705882352941))